# 编解码器模型 Encoder-Decoder Model

随着RNN/LSTM在序列建模中显示出的潜力，研究者开始探索如何将这种能力应用于输入输出都是序列的任务。

2014年，Cho等人提出了基于RNN的编码器-解码器框架用于机器翻译，同年Sutskever等人改进了这一架构并应用于神经机器翻译(NMT)

标志着编解码器模型正式成为序列到序列(Seq2Seq)学习的主流框架。

- 编解码器模型的基本原理

    编解码器模型由两个主要组件构成：
    
    - 编码器(Encoder)：将输入序列（如源语言句子）编码为一个固定维度的上下文向量(context vector)
    
    - 解码器(Decoder)：基于上下文向量生成输出序列（如目标语言句子）
    
    编码阶段：输入序列通过编码器（通常是RNN/LSTM/GRU）逐步处理，最终生成一个浓缩输入信息的上下文向量
    
    状态传递：将上下文向量传递给解码器作为初始状态
    
    解码阶段：解码器逐步生成输出序列，通常使用自回归方式（前一步输出作为当前步输入）


- 编解码器模型解决的核心问题

    变长序列处理：突破了传统神经网络固定输入输出维度的限制，能够处理任意长度的序列数据。
    
    端到端学习：在机器翻译等任务中，取代了复杂的多阶段流水线，实现了从源语言到目标语言的直接映射学习。
    
    语义表示学习：编码器能够将输入序列编码为稠密的分布式表示，捕捉深层语义特征。
    
    跨模态转换：不仅限于文本，还可应用于语音-文本、图像-文本等不同模态间的转换任务。

下面会实现一个简单的编解码器模型

In [1]:
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
class Encoder(torch.nn.Module):
    def __init__(self, input_vs: int, emb_size: int, hidden_size: int, num_layers: int, dropout: float):
        super().__init__()
        self.embedding = torch.nn.Embedding(input_vs, emb_size)
        self.lstm = torch.nn.LSTM(emb_size, hidden_size, num_layers, dropout=dropout, batch_first=True)

    def forward(self, x: torch.Tensor):
        embed = self.embedding(x)
        outputs, state = self.lstm(embed)
        return outputs, state


class Decoder(torch.nn.Module):
    def __init__(self, output_vs: int, emb_size: int, hidden_size: int, num_layers: int, dropout: float):
        super().__init__()
        self.embedding = torch.nn.Embedding(output_vs, emb_size)
        self.lstm = torch.nn.LSTM(emb_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size, output_vs)

    def forward(self, x: torch.Tensor, state: tuple[torch.Tensor, torch.Tensor]):
        embed = self.embedding(x)
        output, state = self.lstm(embed, state)
        decoded = self.fc(output)
        return decoded, state


class EDM(torch.nn.Module):
    def __init__(
        self,
        input_vs: int,
        output_vs: int,
        emb_size: int,
        hidden_size: int,
        num_layers: int,
        dropout: float = 0,
        teacher_forcing_ratio: float = 0.5,
    ):
        super().__init__()
        self.encoder = Encoder(input_vs, emb_size, hidden_size, num_layers, dropout)
        self.decoder = Decoder(output_vs, emb_size, hidden_size, num_layers, dropout)
        self.teacher_forcing_ratio = torch.tensor(teacher_forcing_ratio)

    def forward(self, source: torch.Tensor, target: torch.Tensor):
        _, state = self.encoder(source)
        outputs = []
        x = target[:, 0].unsqueeze(-1)
        for target_t in target.unbind(1):
            output, state = self.decoder(x, state)
            outputs.append(output)
            if torch.rand(1) < self.teacher_forcing_ratio:
                x = target_t.unsqueeze(-1)
            else:
                x = output.argmax(-1)
        return torch.cat(outputs, 1)

In [3]:
model = EDM(100, 200, 50, 64, 2, 0.1)

source = torch.randint(0, 100, (32, 10))
target = torch.randint(0, 100, (32, 15))

outputs = model(source, target)

print(outputs.shape)

torch.Size([32, 15, 200])


# 注意力机制

注意力机制是基于循环神经网络的一种改进。在上面的例子中，我们可以看到编码器的输出完全没有使用。

由于循环神经网络每一层的隐藏状态都对应了输入直到当前的语义，但在最后输出时却只用到了最后一个隐藏状态，这是一种浪费。

而且，隐藏状态每一层都乘以当前权重，几层之后前文的影响会变得非常小。为了解决这个问题，人们首先提出了长短时记忆神经网络。

人们很快意识到，词元的权重其实并不取决于它到当前词元的距离，所以在循环神经网络中，人们把每个词元的隐藏状态经过**加权平均**，形成一个统领全局的语义向量。

这个全局向量就是注意力。

# 注意力机制的实现

输入数据经过编码器时把每一步的隐藏状态记录下来，我们会得到一个 T 个隐藏状态

每个 token 都对应一个编码后的数据**输入隐藏状态**。最后一个隐藏状态是解码器的初始输入，这个是经典循环神经网络的实现机制

经典 RNN 解码器的输入是从空隐藏状态开始逐个输入**输入隐藏状态**，然后输出的隐藏状态传给下个循环

为了解决 RNN 固有的缺陷，人们在此基础上提出了注意力机制

也就是把**全部输入隐藏状态**经过**加权**平均后的结果传进编码器的隐藏状态里，让这个神经网络着重注意更相关的词

# 为什么计算加权和是通过张量矩阵乘法实现的？

这个问题来自于经典RNN注意力机制中的编码器隐藏状态和和解码器隐藏状态进行内积计算对齐分数

在上面的例子里只看编解码器隐藏状态 B 维度里的一个张量 编码器隐藏状态 enc_hids（大小为 [T,H]） 和 当前输入的解码器隐藏状态 dec_hid （大小为 [H]）

*暂令 B=3，H=5*
```
enc_hids = [
    [E11,E12,E13,E14,E15],
    [E21,E22,E23,E24,E25],
    [E31,E32,E33,E34,E35],
]
dec_hid = [D11,D12,D13,D14,D15]
```
enc_hids 的每一行都是编码器 token 的隐藏状态。

设计思路是解码器的隐藏状态应该给和自己更相似的且更重要的 tokrn 更高权重。

因此通常计算内积比较合适。

把隐藏状态看作向量，内积比较大的情况满足两个向量方向相似，并且两个向量乘积本身较大 （类比 力*位移=做功 给做功大的位移的更大权重）

则权重应该如此计算

```
S1 = E11*D11 + E12*D12 + ... + E1n*D1n
S2 = E21*D11 + E22*D12 + ... + E2n*D1n
.
.
.
Sn = En1*D11 + En2*D12 + ... + Enn*D1n
```
dec_hid 和转置的 enc_hids 矩阵相乘刚好能得到权重矩阵
```
[D11,D12,D13,D14,D15] @ [
    [E11,E21,E31],
    [E12,E22,E32],
    [E13,E23,E33],
    [E14,E24,E34],
    [E15,E25,E35],
] = [S1,S2,S3]
```
权重矩阵和 enc_hids 矩阵相乘又刚好是加权结果
```
[S1,S2,S3] @ [
    [E11,E12,E13,E14,E15],
    [E21,E22,E23,E24,E25],
    [E31,E32,E33,E34,E35],
] = [
    S1 * [E11,E12,E13,E14,E15],
    S2 * [E21,E22,E23,E24,E25],
    S3 * [E31,E32,E33,E34,E35],
]
```

# 在解码输出阶段注意力范围应该是全部输入隐藏状态吗？

如果是序列到序列模式，两个 rnn 神经元接收到的输入文本不同，经过各自的 embedding 再建立注意力的话就是**交叉注意力**

但我们的实现是解码器的输入文本是是编码器的输入文本生成的，实际上应该算是**双向自注意力**，因为解码器输入的注意范围是全部输入

但也许应该只让解码器注意当前输入之前的文本，因为 大 名 鼎 鼎 的 Transformer 模型架构 就是这么做的，这种形式叫**单向自注意力**

下面是一段简单的交叉注意力编解码器模型实现。

- *由于注意力机制导致编解码器的耦合程度变大，在下面的实现中我们会将模型组件放进同一个模型里以便更好展示。*

- *若使用 nn.LSTM 依照经典编解码器模型实现注意力机制的方法，应注意 outputs 是多层LSTM最后一层每步的隐藏状态, hiddens 是多层LSTM最后一步的每层的因此隐藏状态。在引入注意力机制时应该手动循环解码器的每一步并记录隐藏状态，并且让编码器每层注意相应层次的隐藏状态*

- *在下面的实现中若解码器的词嵌入维度等于隐藏层维度，那么可以使解码器的 rnn 输入维度等于隐藏层维度从而去掉投影层*

In [4]:
class AttentionEDM(torch.nn.Module):

    def __init__(
        self,
        input_vs: int,
        output_vs: int,
        emb_size: int,
        hidden_size: int,
        num_layers: int,
        dropout: float = 0,
        teacher_forcing_ratio: float = 0.5,
    ):
        super().__init__()
        # 如果两个序列来自同一文本则只需要一个共用的 Embedding
        self.encoder_emb = torch.nn.Embedding(input_vs, emb_size)
        self.encoder_lstm = torch.nn.LSTM(emb_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        # 引入注意力机制需要使解码器词嵌入维度和隐藏层维度相同，或解码器词向量通过投影层转换成隐藏层维度
        self.decoder_emb = torch.nn.Embedding(output_vs, emb_size)
        self.projection = torch.nn.Linear(emb_size, hidden_size)
        self.decoder_lstm = torch.nn.LSTM(emb_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        self.lm = torch.nn.Linear(hidden_size, output_vs)
        self.num_layers = num_layers
        self.teacher_forcing_ratio = torch.tensor(teacher_forcing_ratio)
        # # 如使用单向注意力则需要用 register_buffer 注册一个不会更新的下三角矩阵
        # tril = torch.tril(torch.ones(4096, 4096))
        # self.register_buffer("tril", tril)

    def forward(self, source: torch.Tensor, target: torch.Tensor):
        embed_source: torch.Tensor = self.encoder_emb(source)
        encoder_hiddens = []
        state = None
        for x in embed_source.unbind(1):
            _, (hidden, cell) = self.encoder_lstm(x.unsqueeze(1), state)
            state = (hidden, cell)
            encoder_hiddens.append(hidden)
        encoded = torch.stack(encoder_hiddens, dim=-2)
        encoded_trans = torch.transpose(encoded, -2, -1)
        sqrtT: float = encoded_trans.size(-1) ** 0.5
        embed_target: torch.Tensor = self.decoder_emb(target)
        x = embed_target[:, 0].unsqueeze(1)
        outputs = []
        for time_step, target_t in enumerate(embed_target.unbind(1)):
            scores = self.projection(x).unsqueeze(0) @ encoded_trans
            scores = torch.softmax(scores / sqrtT, dim=-1)
            # # 单向注意力下会使用下三角矩阵将当前时间步之后的权重分数置零
            # scores = scores.masked_fill(self.tril[:time_step, :time_step] == 0, float("-inf"))
            hidden = scores @ encoded
            output, (hidden, cell) = self.decoder_lstm(x, (hidden.squeeze(-2), cell))
            output = self.lm(output)
            outputs.append(output)
            if torch.rand(1) < self.teacher_forcing_ratio:
                x = target_t.unsqueeze(1)
            else:
                x = self.decoder_emb(torch.argmax(output, dim=-1))
        return torch.cat(outputs, 1)

In [5]:
model = AttentionEDM(100, 200, 50, 64, 2, 0.1)
source = torch.randint(0, 100, (32, 10))
target = torch.randint(0, 200, (32, 15))

outputs = model(source, target)

print(outputs.shape)

torch.Size([32, 15, 200])
